# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+3) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 5.10.25 23:59__   
__Жесткий дедлайн: 8.10.25 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('data/biotech_news.tsv', sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


In [2]:
dataset['text'][0]

'drive your plow over the bones of the dead by olga tokarczuk i am an incredibly slow reader but the tone and specificity of the world she creates in this book was something i couldnt leave behind until it was done also all we sawby anne michaels fight nightby miriam toews and the summer before the darkby doris lessing id like turned into a netflix show by amia srinivasan one of the most brain shattering books ive ever read her thinking is so electrically rigorous and fearless i double dare them to make this into a netflix show i last bought i rediscovered her poetry lately and i feel like i dont want to read anything else for a while she owns desire and submerged things has the greatest ending by j d salinger the last page always leaves me breathless the intimacy and truth of that final page is so arresting and almost painful to read should be on every college syllabus by anton piatigorsky a fascinating fictional account of the adolescence of dictators it is painstakingly researched a

## Предобработка лейблов


__Задание 1 (0.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [3]:
ls = dataset['labels'].apply(lambda s: [t.strip() for t in s.split(',') if t.strip()])
C = sorted({t for L in ls for t in L})
c2i = {t:i for i,t in enumerate(C)}

Y = np.zeros((len(ls), len(C)), dtype=np.float32)
for i,L in enumerate(ls):
    for t in L:
        Y[i, c2i[t]] = 1.0
X = dataset['text'].astype(str).tolist()


## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [4]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

__Задание 2 (1 балл)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import Counter

stop = set(ENGLISH_STOP_WORDS)

def tok(s): 
    return [w for w in s.split() if w]

def clean(s): 
    return [w for w in tok(s) if w in ok]

df = Counter()
for s in texts_train: 
    df.update(set(tok(s)))

In [6]:
min_df = 2
rare_words = {w for w, c in df.items() if c < min_df}
print(len(rare_words), list(rare_words)[:100])

max_df = 0.75
n = len(texts_train)
freq_words = {w for w, c in df.items() if c / n > max_df}
print(len(freq_words), list(freq_words)[:100])

19249 ['levante', 'sullivanfrom', 'analystsofficially', '639', 'andria', 'header', 'rw00014506193', 'overtook', 'aliases', 'rxis', 'newcomer', 'notwithstanding', '3trn', 'logjam', 'eleadora', 'laments', 'lm', 'delitunan', 'courtenay', 'diffusers', 'fedohas', 'auntie', 'encoded', 'wineries', 'startengine', 'ontogeny', 'kuerponan', 'bcbst', 'cancels', 'bushy', 'psychophysics', 'myt', 'touc', 'grading', 'booms', 'sigwart', 'interns', 'greenwashed', '2835', 'macrotra', 'smythe', 'lister', 'lyndsey', 'kpop', 'mikhail', 'cottonmouthsis', 'mikhaylovich', 'sidestep', 'quad', 'splend', 'alpro', 'unwelcoming', 'reciprocity', 'tereno', 'ventilatory', 'ians', 'moderno', 'energizer', 'aym', 'honorable', 'sanjiv', 'fisted', 'vodka', 'scripture', 'percelen', 'starwood', 'connor', 'itukraine', '1416', 'oligopoly', 'slurs', 'skyscraper', 'blues', 'barcelonas', 'clutches', 'bixby', 'falk', 'nortis', 'nursesto', 'alkeon', 'corzine', 'categorys', 'millionchildrenhave', 'chinking', 'blakely', 'glovis', 'ca

In [7]:
min_df, max_df = 2, 0.75
ok = {w for w, c in df.items() 
      if c >= min_df and c/n <= max_df and w not in stop and w.isalpha() and len(w)>1}

tr_tok = [clean(s) for s in texts_train]
te_tok = [clean(s) for s in texts_test]

__Задание 3 (1.5 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +3 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [8]:
from tqdm import tqdm

class BPE:
    def __init__(self, vs):
        self.vs = vs
        self.m = []
        self.itos = []
        self.stoi = {}
        self.pad = '<pad>'
        self.unk = '<unk>'

    def _merge_seq(self, seq, a, b):
        out, i, n, ab = [], 0, len(seq), a + b
        while i < n:
            if i + 1 < n and seq[i] == a and seq[i+1] == b:
                out.append(ab)
                i += 2
            else:
                out.append(seq[i])
                i += 1
        return out

    def _merge_multi(self, seq, pairs):
        P = {p: (p[0] + p[1]) for p in pairs}
        out, i, n = [], 0, len(seq)
        while i < n:
            if i + 1 < n:
                a, b = seq[i], seq[i+1]
                if (a, b) in P:
                    out.append(P[(a, b)])
                    i += 2
                    continue
            out.append(seq[i])
            i += 1
        return out

    def fit(self, X):
        S = [list(s) for s in X]
        base = sorted({ch for s in S for ch in s})
        k = max(0, self.vs - 2 - len(base))
        self.m = []
        done = 0
        pbar = tqdm(total=k)
        seen = set()
        while done < k:
            cnt = Counter()
            for seq in S:
                for a, b in zip(seq, seq[1:]):
                    if b != ' ':
                        cnt[(a, b)] += 1
            if not cnt:
                break
            want = int(np.sqrt(k - done))
            pairs = []
            for (a, b), _ in cnt.most_common():
                if (a, b) in seen:
                    continue
                pairs.append((a, b))
                if len(pairs) == want:
                    break
            if not pairs:
                break
            self.m.extend(pairs)
            seen.update(pairs)
            S = [self._merge_multi(seq, pairs) for seq in S]
            done += len(pairs)
            pbar.update(len(pairs))
        pbar.close()
        self.itos = [self.pad, self.unk] + base + [a + b for a, b in self.m]
        self.stoi = {t: i for i, t in enumerate(self.itos)}

    def enc(self, s):
        seq = list(s)
        for a, b in self.m:
            seq = self._merge_seq(seq, a, b)
        return [self.stoi.get(t, 1) for t in seq]

    def enc_batch(self, X):
        return [self.enc(s) for s in X]


In [9]:
bpe = BPE(vs=3000)
bpe.fit(texts_train)
PAD = 0
V = len(bpe.itos)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2960/2960 [02:35<00:00, 19.02it/s]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

Xtr = [bpe.enc(s) for s in tqdm(texts_train)]
Xte = [bpe.enc(s) for s in tqdm(texts_test)]

class DS(Dataset):
    def __init__(self, X, Y): 
        self.X, self.Y = X, torch.tensor(Y, dtype=torch.float32)

    def __len__(self): 
        return len(self.X)
    
    def __getitem__(self, i): 
        return self.X[i], self.Y[i]

MAX_L = 600

def collate(b):
    xs, ys = zip(*b)
    xs = [x[:MAX_L] for x in xs]
    ln = torch.tensor([len(x) for x in xs], dtype=torch.long)
    L = int(ln.max())
    pad = torch.full((len(xs), L), PAD, dtype=torch.long)
    for i,x in enumerate(tqdm(xs, leave=False)):
        if x:
            pad[i,:len(x)] = torch.tensor(x, dtype=torch.long)
    return pad, ln, torch.stack(ys)


tr_ds = DS(Xtr, y_train)
te_ds = DS(Xte, y_test)

tr_dl = DataLoader(
    tr_ds, 
    batch_size=64, 
    shuffle=True, 
    collate_fn=collate
)
te_dl = DataLoader(
    te_ds, 
    batch_size=64, 
    shuffle=False, 
    collate_fn=collate
)


  7%|██████████▋                                                                                                                                                    | 163/2431 [00:45<10:13,  3.69it/s]

In [ ]:
if torch.backends.mps.is_available():
    dev = torch.device('mps')
elif torch.cuda.is_available():
    dev = torch.device('cuda')
else:
    dev = torch.device('cpu')
    
torch.manual_seed(0)
print(dev)

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [ ]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape
    return f1_score(y_true, y_pred, average='macro', zero_division=0)

def best_thr(y_true, y_prob, grid=np.linspace(0.05, 0.6, 24)):
    f1s = [compute_f1(y_true, (y_prob >= t).astype(np.int32)) for t in grid]
    i = int(np.argmax(f1s))
    return float(grid[i]), float(f1s[i])


## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [13]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, V, E, H, O, pad=0):
        super().__init__()
        self.emb = nn.Embedding(V, E, padding_idx=pad)
        self.W = nn.Linear(E, H, bias=True)
        self.U = nn.Linear(H, H, bias=False)
        self.o = nn.Linear(H, O)

    def forward(self, x, ln):
        e = self.emb(x)
        B,T,_ = e.shape
        h = torch.zeros(B, self.U.out_features, device=e.device)
        Hs = []
        for t in range(T):
            h = torch.tanh(self.W(e[:,t]) + self.U(h))
            Hs.append(h)
        Hs = torch.stack(Hs, 1)
        idx = (ln-1).view(-1,1,1).expand(-1,1,Hs.size(-1))
        last = Hs.gather(1, idx).squeeze(1)
        return self.o(last)


In [ ]:
pw = torch.tensor((y_train.shape[0] - y_train.sum(0)) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=dev)

def fit(m, tr, te, ep=10, lr=3e-3, wd=1e-5, l1_w=1e-5, clip=1.0, pos_weight=None):
    m.to(dev)
    crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight) if pos_weight is not None else nn.BCEWithLogitsLoss()
    opt = torch.optim.Adam(m.parameters(), lr=lr, weight_decay=wd)  # wd это L2
    
    for e in range(ep):
        m.train()
        tot, n = 0.0, 0
        pbar = tqdm(tr, desc=f'train {e+1}/{ep}', leave=False)
        for xb, lb, yb in pbar:
            xb, lb, yb = xb.to(dev), lb.to(dev), yb.to(dev)
            opt.zero_grad()
            z = m(xb, lb)
            main_loss = crit(z, yb)
            
            l1_loss = 0.0
            if l1_w > 0:
                for param in m.parameters():
                    l1_loss += torch.norm(param, 1)
            
            total_loss = main_loss + l1_w * l1_loss
            
            total_loss.backward()
            nn.utils.clip_grad_norm_(m.parameters(), clip)
            opt.step()
            
            bs = xb.size(0)
            tot += total_loss.item() * bs
            n += bs
            pbar.set_postfix(loss=f'{tot/n:.4f}')
        
        m.eval()
        yp, yt = [], []
        with torch.no_grad():
            for xb, lb, yb in te:
                xb, lb = xb.to(dev), lb.to(dev)
                p = torch.sigmoid(m(xb, lb)).cpu().numpy()
                yp.append(p)
                yt.append(yb.numpy().astype(np.int32))
        yp = np.vstack(yp)
        yt = np.vstack(yt)
        thr, f1 = best_thr(yt, yp)
        print(f'ep {e+1:02d} thr {thr:.3f} f1 {f1:.4f}')
    
    return m


In [25]:
E, H = 64, 128
rnn = RNN(V, E, H, y_train.shape[1], pad=PAD)
rnn = fit(rnn, tr_dl, te_dl, ep=30, pos_weight=pw)

train 1/30:   0%|          | 0/38 [00:00<?, ?it/s]

ep 01 thr 0.480 f1 0.1089


ep 02 thr 0.480 f1 0.1112


ep 03 thr 0.504 f1 0.1215


ep 04 thr 0.457 f1 0.1226


...

ep 26 thr 0.600 f1 0.2921


ep 27 thr 0.600 f1 0.2898


ep 28 thr 0.600 f1 0.2818


ep 29 thr 0.600 f1 0.2514


ep 30 thr 0.600 f1 0.2591


### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [16]:
class LSTMCell(nn.Module):
    def __init__(self, I, H):
        super().__init__()
        self.W = nn.Linear(I, 4*H, bias=True)
        self.U = nn.Linear(H, 4*H, bias=False)
        self.H = H
    def forward(self, x, h, c):
        g = self.W(x) + self.U(h)
        f,i,gc,o = torch.chunk(g, 4, -1)
        f,i,o = torch.sigmoid(f), torch.sigmoid(i), torch.sigmoid(o)
        gc = torch.tanh(gc)
        c = f*c + i*gc
        h = o*torch.tanh(c)
        return h,c

class LSTM(nn.Module):
    def __init__(self, V, E, H, O, pad=0):
        super().__init__()
        self.emb = nn.Embedding(V, E, padding_idx=pad)
        self.cell = LSTMCell(E, H)
        self.o = nn.Linear(H, O)
        self.H = H
    def forward(self, x, ln):
        e = self.emb(x)
        B,T,_ = e.shape
        h = torch.zeros(B, self.H, device=e.device)
        c = torch.zeros(B, self.H, device=e.device)
        Hs = []
        for t in range(T):
            h,c = self.cell(e[:,t], h, c)
            Hs.append(h)
        Hs = torch.stack(Hs,1)
        idx = (ln-1).view(-1,1,1).expand(-1,1,self.H)
        last = Hs.gather(1, idx).squeeze(1)
        return self.o(last)


In [26]:
H2 = 112
lstm = LSTM(V, E, H2, y_train.shape[1], pad=PAD)
lstm = fit(lstm, tr_dl, te_dl, ep=30, pos_weight=pw)


ep 01 thr 0.480 f1 0.1062


ep 02 thr 0.457 f1 0.1073


ep 03 thr 0.457 f1 0.1095


ep 04 thr 0.504 f1 0.1244


ep 05 thr 0.433 f1 0.1311


...

ep 26 thr 0.600 f1 0.2948


ep 27 thr 0.600 f1 0.2943


ep 28 thr 0.600 f1 0.2844


ep 29 thr 0.528 f1 0.2906


ep 30 thr 0.600 f1 0.2866


__Задание 6 (2 балла).__ Главный недостаток RNN моделей заключается в том, что при сжатии всей информации в один вектор, важные детали пропадают. Для решения этой проблемы был придуман механизм внимания. Реализуйте его по [оригинальной статье](https://arxiv.org/abs/1409.0473). Замерьте качество и сделайте выводы.   
Обратите внимание, что метод был предложен для Encoder-Decoder моделей. В нашем случае декодера нет, поэтому встройте внимание в энкодер: каждый блок LSTM будет смотреть на выходы всех предыдущих блоков.   

In [18]:
class LSTMA(nn.Module):
    def __init__(self, V, E, H, O, pad=0):
        super().__init__()
        self.emb = nn.Embedding(V, E, padding_idx=pad)
        self.cell = LSTMCell(E, H)

        self.Wk = nn.Linear(H, H, bias=False)
        self.Wq = nn.Linear(H, H, bias=False)
        self.v = nn.Linear(H, 1, bias=False)

        self.Wc = nn.Linear(2*H, H)
        self.o = nn.Linear(H, O)
        self.H = H
        
    def forward(self, x, ln):
        e = self.emb(x)
        B,T,_ = e.shape
        h = torch.zeros(B, self.H, device=e.device)
        c = torch.zeros(B, self.H, device=e.device)
        Hs = []
        for t in range(T):
            h,c = self.cell(e[:,t], h, c)
            Hs.append(h)
        H = torch.stack(Hs,1)
        idx = (ln-1).clamp_min(0).view(-1,1,1).expand(-1,1,self.H)
        hT = H.gather(1, idx).squeeze(1)
        K = self.Wk(H)
        q = self.Wq(hT).unsqueeze(1)
        s = self.v(torch.tanh(K + q)).squeeze(-1)
        mask = (torch.arange(T, device=e.device).unsqueeze(0) >= ln.unsqueeze(1))
        s = s.masked_fill(mask, float('-inf'))
        a = torch.softmax(s, -1)
        ctx = (a.unsqueeze(-1) * H).sum(1)
        h_ = torch.tanh(self.Wc(torch.cat([hT, ctx], -1)))
        return self.o(h_)


In [27]:
lstm_a = LSTMA(V, E, H2, y_train.shape[1], pad=PAD)
lstm_a = fit(lstm_a, tr_dl, te_dl, ep=30, pos_weight=pw)

ep 01 thr 0.480 f1 0.1065


ep 02 thr 0.480 f1 0.1091


ep 03 thr 0.504 f1 0.1234


ep 04 thr 0.480 f1 0.1299


ep 05 thr 0.528 f1 0.1703


...

ep 26 thr 0.600 f1 0.3206


ep 27 thr 0.600 f1 0.3361


ep 28 thr 0.600 f1 0.3242


ep 29 thr 0.600 f1 0.3234


ep 30 thr 0.600 f1 0.3473


__Задание 7 (1 балл).__ Добавьте в вашу реализации возможность увеличивать число слоев LSTM. Обучите модель с двумя слоями и замерьте качество. Сделайте выводы: стоит ли увеличивать размер модели?

Лень модифицировать, но мое предположение в том, что на таком маленьком датасете увеличение размера модели скорее увеличит риск переобучения, а не качество. Хотя, конечно, если запариться, можно немного улучшить качество. Наверняка, случайно выбранные параметры не самые оптимальные.